# Keras 

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


In [0]:
num_classes=10
iterations=int(100)
learning_rate=0.001
batch_size=128

conv1_filters=32
conv2_filters=64
fc1_units=1024


# prepare mnist data

In [0]:
from tensorflow.keras.datasets import  mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [0]:
x_train,x_test=np.array(x_train,np.float32),np.array(x_test,np.float32)
x_train,x_test=x_train/255,x_test/255

# use tf.data to shuffle and batch data

In [0]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train)) # x_train ra y_train lai suffal ta garxa tara corrosponding label sangai shuffle garxa

In [0]:
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)


In [0]:
def conv2d(x,w,b,strides=1):
  x=tf.nn.conv2d(x,w,strides=[1,strides,strides,1],padding='SAME')
  x=tf.nn.bias_add(x,b)
  return tf.nn.relu(x)


In [0]:
def maxpool2d(x,k=2):
  return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k],padding='SAME')
  

In [0]:
random_normal=tf.initializers.RandomNormal()
weights={
    'c1':tf.Variable(random_normal([5,5,1,conv1_filters])),
    'c2':tf.Variable(random_normal([5,5,conv1_filters,conv2_filters])),
    'f1':tf.Variable(random_normal([7*7*64,fc1_units])),
    'out':tf.Variable(random_normal([fc1_units,num_classes]))



}

In [0]:
biases={
    'bc1':tf.Variable(tf.zeros([conv1_filters])),
    'bc2':tf.Variable(tf.zeros([conv2_filters])),
    'bfc1':tf.Variable(tf.zeros([fc1_units])),
    'bout':tf.Variable(tf.zeros([num_classes]))
}

In [0]:
def conv_net(x):
  x=tf.reshape(x,[-1,28,28,1])
  conv1=conv2d(x,weights['c1'],biases['bc1'])
  conv1=maxpool2d(conv1,k=2)
  conv2=conv2d(x,weights['c2'],biases['bc2'])
  conv2=maxpool2d(conv2,k=2)
  fc1=tf.reshape(conv2,[-1,weights['fc'].get_shape().as_list()[0]])
  fc1=tf.add(tf.matmul(fc1,weights['fc'],biases['bfc1']))
  fc1=tf.nn.relu(fc1)
  out=tf.add(tf.matmul(fc1,weights['out']),biases['bout'])
  return tf.nn.softmax(out)
  

In [0]:
def cross_entropy(y_pred,y_true):
  y_true=tf.one_hot(y_true,depth=num_classes)
  y_pred=tf.clip_by_value(y_pred,1e-9,1.)
  return tf.reduce_mean(tf.cast(correct_prediction,tf.float32),axis=-1)
  

In [0]:
def accuracy(y_pred,y_true):
  correct_prediction=tf.equal(tf.argmax(y_pred,1),tf.cast(y_true,tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction,tg.float32),axis=-1)

In [0]:
optimizer=tf.optimizers.Adam(learning_rate)

In [0]:
trainable_weights=[]
for i in weights:
  trainable_weights.append(weights[i])
for i in  biases:
  trainable_weights.append(biases[i])

In [0]:
def train(x,y,trainable_weights):
  with tf.GradientTape() as g:
    pred=conv_net(x)
    loss=cross_entropy(pred,y)
  gradients=g.gradient(loss,trainable_weights)
  optimizer.apply_gradients(zip(gradient,trainable_weights))


In [108]:
for step,(batch_x,batch_y) in  enumerate(train_data.take(iterations),i):
  train(batch_x,batch_y,trainable_weights)
  if step%10==0:
    pred=conv_net(batch_x)
    loss=cross_entropy(pred,batch_y)
    acc=accuracy(pred,batc_y)
    print('step %i,loss:%f , accurcy: %f' %(step,loss,acc))

TypeError: ignored